In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Preparacion de Datos

In [2]:
# Update the paths to point to your image directories with forward slashes
base_dir = r'C:/Users/diego/Desktop/TSCDIA/Modelizado de Sistemas de IA/Mod git/Modelizado-main/image_set'  # Base directory
train_dir = base_dir + r'/train'
val_dir = base_dir + r'/val'
test_dir = base_dir + r'/test'

In [3]:
# Image data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')


In [4]:
# Validation and test data generators without augmentation
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Load the training data with target size set to 256x256 (since your images are already that size)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(256, 256), 
                                                    batch_size=32, class_mode='categorical')


In [ ]:
# Load the validation data with target size 256x256
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(256, 256), 
                                                batch_size=32, class_mode='categorical')


## Entrenamiento del Modelo

In [8]:
# Load the Xception model as a base (pretrained on ImageNet)
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models


In [9]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # Freeze the pretrained model layers


In [10]:
# Build the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Display the model summary
model.summary()


Le subimos las epocas a 50 con un early stop de 3, (se puede subir a mas para tener mejor precision por lo que parece)

In [ ]:
# Train the model
#history = model.fit(train_generator, epochs=5, validation_data=val_generator)

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator, epochs=50, validation_data=val_generator, 
                    callbacks=[early_stopping])

In [ ]:
# Load and evaluate on test data with target size 256x256
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(256, 256), 
                                                  batch_size=32, class_mode='categorical')


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


## Guardar/Cargar el modelo

In [ ]:
# guardar el modelo
# model.save('C:/Users/diego/Desktop/TSCDIA/Modelizado de Sistemas de IA/Modelizado/Modelo/vehiculos.keras')

# cargar el modelo
from tensorflow.keras.models import load_model
model = load_model('C:/Users/diego/Desktop/TSCDIA/Modelizado de Sistemas de IA/Mod git/Modelizado-main/Modelo/vehiculos.keras')


## Predicciones

In [8]:
# Predict on a single image
from tensorflow.keras.preprocessing import image
import numpy as np


In [9]:
img_path = r'C:/Users/diego/Downloads/download.jpg'  # Update this path
img = image.load_img(img_path, target_size=(256, 256))  # Adjust to 256x256
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)


In [ ]:
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
print(f'Predicted class: {predicted_class}')


In [ ]:
label_list = train_generator.class_indices
# Invertimos el diccionario para que podamos buscar los nombres de las clases por índice
label_list = {v: k for k, v in label_list.items()}
predicted_label = label_list[predicted_class]
print(f'This vehicle is predicted to be a: {predicted_label}')


## Flask

In [ ]:
from flask import Flask, request, render_template
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == "POST":
        # Cargar la imagen del formulario
        file = request.files['file']
        img_path = './uploads/' + file.filename
        file.save(img_path)
        
        # Cargar y preprocesar la imagen
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Hacer predicción
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions[0])
        
        # Convertir índice a clase
        predicted_label = label_list[predicted_class]
        return f'El vehículo es: {predicted_label}'
    return '''
    <!doctype html>
    <title>Subir una imagen</title>
    <h1>Subir una imagen para clasificar el vehículo</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Subir>
    </form>
    '''

if __name__ == "__main__":
    import os
    # Evitar la ejecución en entornos interactivos como Jupyter/IPython
    if os.environ.get("FLASK_ENV") != "development":
        app.run(debug=True)
